# Data Cleaning & Preprocessing

This notebook prepares the online gambling comment dataset for conventional (e.g., TF-IDF + Linear/Tree models) and modern pipelines (e.g., transformer embeddings). It follows the EDA guide's priorities, with a focus on stylized Unicode handling and class imbalance.

## Plan
- Load raw `train.csv`, `test.csv`, `holdout.csv` with delimiter auto-detection and consistent schema.
- Normalize stylized Unicode and consolidate spaced stylized sequences (highest priority).
- Clean text while preserving discriminative signals; add feature flags for downstream use.
- Create balanced 50:50 splits for each dataset using stratified sampling/oversampling.
- Export cleaned and balanced datasets to `processed_data/` as CSV and pickle.
- Provide quick validation summaries.

Outputs:
- `processed_data/*_processed.csv|pkl` (original distribution, cleaned)
- `processed_data/*_balanced_50_50.csv|pkl` (balanced by label)
- `processed_data/all_datasets.pkl` (aggregated)


In [22]:
# Imports and setup
import os
import re
import sys
import json
import math
import unicodedata
from typing import Tuple, List

import numpy as np
import pandas as pd

from collections import Counter

PROCESSED_DIR = "processed_data"
RAW_DIR = "dataset"
RANDOM_STATE = 42

os.makedirs(PROCESSED_DIR, exist_ok=True)


In [23]:
# Loader with delimiter detection
import csv
from io import StringIO


def read_csv_auto(path: str) -> pd.DataFrame:
    with open(path, 'r', encoding='utf-8') as f:
        head = f.read(2048)
    # Try csv.Sniffer on small sample
    try:
        dialect = csv.Sniffer().sniff(head, delimiters=[',',';','\t'])
        sep = dialect.delimiter
    except Exception:
        # Fallback: detect by header token count
        first_line = head.splitlines()[0]
        if ';' in first_line and first_line.count(';') >= first_line.count(','):
            sep = ';'
        else:
            sep = ','
    df = pd.read_csv(path, sep=sep, encoding='utf-8')
    # Normalize column names
    df.columns = [c.strip().lower() for c in df.columns]
    # Ensure expected schema
    rename_map = {}
    if 'comment' not in df.columns:
        # Try common variants
        for c in df.columns:
            if 'comment' in c:
                rename_map[c] = 'comment'
                break
    if 'label' not in df.columns:
        for c in df.columns:
            if 'label' in c or c in {'target','y'}:
                rename_map[c] = 'label'
                break
    if rename_map:
        df = df.rename(columns=rename_map)
    assert {'comment','label'}.issubset(set(df.columns)), f"Missing required columns in {path}: {df.columns}"
    # Coerce types
    df['comment'] = df['comment'].astype(str)
    df['label'] = pd.to_numeric(df['label'], errors='coerce').fillna(-1).astype(int)
    # Drop rows with invalid labels
    df = df[df['label'].isin([0,1])].reset_index(drop=True)
    return df


In [24]:
# Unicode blocks and stylized normalization utilities
# Ranges for mathematical alphanumeric symbols
MATH_ALNUM_RANGES = [
    (0x1D400, 0x1D7FF),  # Mathematical Alphanumeric Symbols
]

FIRE_SYMBOLS = {"🔥", "💥", "⚡"}
MONEY_SYMBOLS = {"💰", "💵", "💸", "🪙", "💷", "💶", "💴", "₿"}


def is_math_alnum_char(ch: str) -> bool:
    cp = ord(ch)
    return any(start <= cp <= end for start, end in MATH_ALNUM_RANGES)


def normalize_unicode_to_ascii(text: str) -> str:
    # Convert to NFKD then strip combining marks
    decomposed = unicodedata.normalize('NFKD', text)
    filtered = ''.join(c for c in decomposed if not unicodedata.combining(c))
    return filtered


def consolidate_spaced_stylized(text: str) -> str:
    # Collapse sequences like "𝐃 𝐎 𝙍 𝘈" -> "𝐃𝐎𝙍𝘈"
    tokens = text.split()
    if len(tokens) <= 1:
        return text
    # Heuristic: if most tokens are single char and many are math alnum, join them
    single_char_tokens = [t for t in tokens if len(t) == 1]
    if len(single_char_tokens) >= max(3, int(0.6 * len(tokens))):
        # join without spaces
        return ''.join(tokens)
    return text


def map_math_alnum_to_base(ch: str) -> str:
    # Map mathematical alphanumeric variants to their base ASCII letter/number when possible
    name = unicodedata.name(ch, '')
    # Examples: MATHEMATICAL BOLD CAPITAL A -> A
    match = re.search(r"MATHEMATICAL (?:BOLD|ITALIC|BOLD ITALIC|SCRIPT|BOLD SCRIPT|FRAKTUR|DOUBLE-STRUCK|SANS-SERIF|SANS-SERIF BOLD|SANS-SERIF ITALIC|SANS-SERIF BOLD ITALIC|MONOSPACE) (CAPITAL|SMALL) ([A-Z])", name)
    if match:
        case, letter = match.groups()
        return letter if case == 'CAPITAL' else letter.lower()
    # Digits
    if name.startswith('MATHEMATICAL DOUBLE-STRUCK DIGIT') or name.startswith('MATHEMATICAL SANS-SERIF DIGIT'):
        # Get last number in name
        digits = re.findall(r"\d", name)
        if digits:
            return digits[-1]
    return ch


def fold_math_alnum(text: str) -> str:
    return ''.join(map_math_alnum_to_base(ch) if is_math_alnum_char(ch) else ch for ch in text)


def normalize_stylized_text(text: str) -> Tuple[str, dict]:
    # Consolidate spaced stylized sequences, then fold math alnum to base
    consolidated = consolidate_spaced_stylized(text)
    folded = fold_math_alnum(consolidated)
    # Keep a plain ASCII fallback for models that need it
    ascii_fallback = normalize_unicode_to_ascii(folded)
    features = {
        'has_math_alnum': any(is_math_alnum_char(c) for c in text),
        'money_symbol_count': sum(c in MONEY_SYMBOLS for c in text),
        'fire_symbol_count': sum(c in FIRE_SYMBOLS for c in text),
    }
    return ascii_fallback, features


In [25]:
# Cleaning and feature flags
URL_RE = re.compile(r"https?://\S+|www\.\S+", re.IGNORECASE)
MENTION_RE = re.compile(r"@[\w_]+", re.UNICODE)
HASHTAG_RE = re.compile(r"#[\w_]+", re.UNICODE)
REPEAT_CHAR_RE = re.compile(r"(.)\1{2,}")
WHITESPACE_RE = re.compile(r"\s+")


def clean_text_preserve_signals(text: str) -> Tuple[str, dict]:
    original = text
    # Remove urls, mentions, hashtags
    text = URL_RE.sub(" ", text)
    text = MENTION_RE.sub(" ", text)
    text = HASHTAG_RE.sub(" ", text)
    # Consolidate stylized sequences and fold math alnum
    norm_ascii, stylized_feats = normalize_stylized_text(text)
    # Lowercase for normalization variants
    lower = norm_ascii.lower()
    # Collapse repeated characters to max 2
    lower = REPEAT_CHAR_RE.sub(r"\1\1", lower)
    # Normalize whitespace
    lower = WHITESPACE_RE.sub(" ", lower).strip()
    # Feature flags
    total_chars = max(len(original), 1)
    num_digits = sum(ch.isdigit() for ch in original)
    num_special = sum((not ch.isalnum()) and not ch.isspace() for ch in original)
    stylized_count = sum(is_math_alnum_char(ch) for ch in original)
    features = {
        **stylized_feats,
        'special_char_ratio': num_special / total_chars,
        'number_ratio': num_digits / total_chars,
        'stylized_char_ratio': stylized_count / total_chars,
        'char_count': len(original),
        'word_count': len(original.split()),
    }
    return lower, features


In [26]:
# Processing functions

def apply_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    cleaned = []
    feats = []
    for txt in df['comment'].astype(str).tolist():
        c, f = clean_text_preserve_signals(txt)
        cleaned.append(c)
        feats.append(f)
    feat_df = pd.DataFrame(feats)
    out = df.copy()
    out['comment_clean'] = cleaned
    for col in feat_df.columns:
        out[col] = feat_df[col].values
    # Additional length features
    out['avg_word_length'] = out['char_count'] / out['word_count'].replace(0, np.nan)
    out['is_very_short'] = out['char_count'] <= 3
    return out


def balance_50_50(df: pd.DataFrame, random_state: int = RANDOM_STATE) -> pd.DataFrame:
    # Oversample minority to match majority; then downsample to exact 50:50 of total
    labels = df['label']
    count0 = int((labels == 0).sum())
    count1 = int((labels == 1).sum())
    if count0 == 0 or count1 == 0:
        return df.copy()
    # Oversample minority to majority
    if count0 > count1:
        maj_df, min_df = df[labels == 0], df[labels == 1]
    else:
        maj_df, min_df = df[labels == 1], df[labels == 0]
    reps = math.ceil(len(maj_df) / len(min_df))
    min_upsampled = pd.concat([min_df.sample(len(min_df), replace=True, random_state=random_state + i) for i in range(reps)], ignore_index=True).iloc[:len(maj_df)]
    balanced = pd.concat([maj_df, min_upsampled], ignore_index=True)
    # Shuffle
    balanced = balanced.sample(frac=1.0, random_state=random_state).reset_index(drop=True)
    return balanced


def save_outputs(df: pd.DataFrame, stem: str):
    csv_path = os.path.join(PROCESSED_DIR, f"{stem}.csv")
    pkl_path = os.path.join(PROCESSED_DIR, f"{stem}.pkl")
    df.to_csv(csv_path, index=False)
    df.to_pickle(pkl_path)
    print(f"Saved: {csv_path} | {pkl_path}")


In [27]:
# Run processing for train/test/holdout
train_df = read_csv_auto(os.path.join(RAW_DIR, 'train.csv'))
test_df = read_csv_auto(os.path.join(RAW_DIR, 'test.csv'))
holdout_df = read_csv_auto(os.path.join(RAW_DIR, 'holdout.csv'))

print('Shapes (raw):', train_df.shape, test_df.shape, holdout_df.shape)

train_proc = apply_cleaning(train_df)
test_proc = apply_cleaning(test_df)
holdout_proc = apply_cleaning(holdout_df)

save_outputs(train_proc, 'train_processed')
save_outputs(test_proc, 'test_processed')
save_outputs(holdout_proc, 'holdout_processed')


Shapes (raw): (8171, 2) (2335, 2) (1167, 2)
Saved: processed_data/train_processed.csv | processed_data/train_processed.pkl
Saved: processed_data/test_processed.csv | processed_data/test_processed.pkl
Saved: processed_data/holdout_processed.csv | processed_data/holdout_processed.pkl


In [28]:
# Create 50:50 balanced splits and save
train_bal = balance_50_50(train_proc, RANDOM_STATE)
test_bal = balance_50_50(test_proc, RANDOM_STATE)
holdout_bal = balance_50_50(holdout_proc, RANDOM_STATE)

print('Label counts (train before/after):', train_proc['label'].value_counts().to_dict(), train_bal['label'].value_counts().to_dict())
print('Label counts (test before/after):', test_proc['label'].value_counts().to_dict(), test_bal['label'].value_counts().to_dict())
print('Label counts (holdout before/after):', holdout_proc['label'].value_counts().to_dict(), holdout_bal['label'].value_counts().to_dict())

save_outputs(train_bal, 'train_balanced_50_50')
save_outputs(test_bal, 'test_balanced_50_50')
save_outputs(holdout_bal, 'holdout_balanced_50_50')


Label counts (train before/after): {0: 7454, 1: 717} {1: 7454, 0: 7454}
Label counts (test before/after): {0: 2126, 1: 209} {0: 2126, 1: 2126}
Label counts (holdout before/after): {0: 942, 1: 225} {1: 942, 0: 942}
Saved: processed_data/train_balanced_50_50.csv | processed_data/train_balanced_50_50.pkl
Saved: processed_data/test_balanced_50_50.csv | processed_data/test_balanced_50_50.pkl
Saved: processed_data/holdout_balanced_50_50.csv | processed_data/holdout_balanced_50_50.pkl


In [29]:
# Aggregate all processed datasets
all_df = pd.concat([
    train_proc.assign(split='train'),
    test_proc.assign(split='test'),
    holdout_proc.assign(split='holdout')
], ignore_index=True)

all_bal_df = pd.concat([
    train_bal.assign(split='train_balanced'),
    test_bal.assign(split='test_balanced'),
    holdout_bal.assign(split='holdout_balanced')
], ignore_index=True)

all_payload = {
    'processed': all_df,
    'balanced': all_bal_df
}

pd.to_pickle(all_payload, os.path.join(PROCESSED_DIR, 'all_datasets.pkl'))
print('Saved aggregated datasets to processed_data/all_datasets.pkl')


Saved aggregated datasets to processed_data/all_datasets.pkl


In [30]:
# Quick validation summaries

def summarize(df: pd.DataFrame, name: str):
    vc = df['label'].value_counts().to_dict()
    print(f"{name}: n={len(df)} labels={vc}")
    print('Avg chars:', df['char_count'].mean().round(2),
          'Avg words:', df['word_count'].mean().round(2),
          'Stylized ratio mean:', df['stylized_char_ratio'].mean().round(4),
          'Num ratio mean:', df['number_ratio'].mean().round(4),
          'Special ratio mean:', df['special_char_ratio'].mean().round(4))

summarize(train_proc, 'train_processed')
summarize(test_proc, 'test_processed')
summarize(holdout_proc, 'holdout_processed')

summarize(train_bal, 'train_balanced_50_50')
summarize(test_bal, 'test_balanced_50_50')
summarize(holdout_bal, 'holdout_balanced_50_50')


train_processed: n=8171 labels={0: 7454, 1: 717}
Avg chars: 64.41 Avg words: 10.51 Stylized ratio mean: 0.0107 Num ratio mean: 0.0236 Special ratio mean: 0.0269
test_processed: n=2335 labels={0: 2126, 1: 209}
Avg chars: 62.29 Avg words: 10.16 Stylized ratio mean: 0.0117 Num ratio mean: 0.0234 Special ratio mean: 0.0275
holdout_processed: n=1167 labels={0: 942, 1: 225}
Avg chars: 62.44 Avg words: 10.14 Stylized ratio mean: 0.0196 Num ratio mean: 0.0212 Special ratio mean: 0.0301
train_balanced_50_50: n=14908 labels={1: 7454, 0: 7454}
Avg chars: 56.53 Avg words: 10.24 Stylized ratio mean: 0.0591 Num ratio mean: 0.035 Special ratio mean: 0.0262
test_balanced_50_50: n=4252 labels={0: 2126, 1: 2126}
Avg chars: 55.39 Avg words: 9.92 Stylized ratio mean: 0.0615 Num ratio mean: 0.0362 Special ratio mean: 0.0254
holdout_balanced_50_50: n=1884 labels={1: 942, 0: 942}
Avg chars: 52.41 Avg words: 8.76 Stylized ratio mean: 0.0478 Num ratio mean: 0.0246 Special ratio mean: 0.0257
